In [1]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. Ultralytics(YOLOv8) 설치 및 필요 라이브러리 임포트
!pip install ultralytics -q
!pip install opencv-python-headless -q

import os
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from ultralytics import YOLO

# GPU 확인
print(f"Setup Complete. Using GPU: {torch.cuda.get_device_name(0)}")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Setup Complete. Using GPU: NVIDIA A100-SXM4-80GB


In [6]:
import shutil
import os

base_dir = '/content/drive/MyDrive/cv2/Laboro_Tomato'
train_label_dir = os.path.join(base_dir, 'train/labels')
val_label_dir = os.path.join(base_dir, 'val/labels')

# 폴더가 있다면 삭제하고 다시 깨끗하게 생성
if os.path.exists(train_label_dir):
    shutil.rmtree(train_label_dir)
os.makedirs(train_label_dir, exist_ok=True)

if os.path.exists(val_label_dir):
    shutil.rmtree(val_label_dir)
os.makedirs(val_label_dir, exist_ok=True)

print("✅ 기존 라벨 폴더를 초기화했습니다. 이제 깨끗합니다.")

✅ 기존 라벨 폴더를 초기화했습니다. 이제 깨끗합니다.


In [7]:
import json
import numpy as np
from tqdm import tqdm

CLASSES = [
    'b_fully_ripened', 'b_half_ripened', 'b_green',
    'l_fully_ripened', 'l_half_ripened', 'l_green'
]

def convert_coco_to_yolo_seg_clean(json_file, save_dir):
    with open(json_file, 'r') as f:
        data = json.load(f)

    images = {item['id']: item for item in data['images']}
    cat_map = {}
    for cat in data['categories']:
        if cat['name'] in CLASSES:
            cat_map[cat['id']] = CLASSES.index(cat['name'])

    for ann in tqdm(data['annotations'], desc=f"Converting {os.path.basename(json_file)}"):
        image_id = ann['image_id']
        if image_id not in images: continue

        img_info = images[image_id]
        img_w = img_info['width']
        img_h = img_info['height']

        file_name = os.path.splitext(img_info['file_name'])[0]
        txt_path = os.path.join(save_dir, file_name + '.txt')

        class_id = cat_map.get(ann['category_id'])
        if class_id is None: continue

        # --- 핵심: Segmentation 데이터가 있는 경우만 처리 ---
        if 'segmentation' in ann and len(ann['segmentation']) > 0:
            for seg in ann['segmentation']:
                poly = np.array(seg).reshape(-1, 2)
                poly[:, 0] /= img_w
                poly[:, 1] /= img_h
                poly = poly.flatten().tolist()

                # 점이 3개 미만(데이터 길이 < 6)인 잘못된 폴리곤은 패스
                if len(poly) < 6: continue

                poly_str = ' '.join([f"{x:.6f}" for x in poly])

                with open(txt_path, 'a') as f:
                    f.write(f"{class_id} {poly_str}\n")

        # 주의: else 문을 제거하여 Box 좌표가 아예 생성되지 않도록 함
        # (YOLO-seg 학습에는 Box 좌표만 있는 라인은 필요 없음)

# 변환 실행
json_path_train = os.path.join(base_dir, 'annotations/train.json')
json_path_test = os.path.join(base_dir, 'annotations/test.json')

print("Train 라벨 재생성 중...")
convert_coco_to_yolo_seg_clean(json_path_train, train_label_dir)

print("Val 라벨 재생성 중...")
if os.path.exists(json_path_test):
    convert_coco_to_yolo_seg_clean(json_path_test, val_label_dir)

print("✅ 라벨 재생성 완료 (중복 및 Box 좌표 제거됨)")

Train 라벨 재생성 중...


Converting train.json: 100%|██████████| 7781/7781 [00:22<00:00, 349.14it/s]


Val 라벨 재생성 중...


Converting test.json: 100%|██████████| 1996/1996 [00:05<00:00, 341.78it/s]

✅ 라벨 재생성 완료 (중복 및 Box 좌표 제거됨)


In [8]:
import glob

# 아무 txt 파일이나 하나 가져오기
sample_files = glob.glob(os.path.join(train_label_dir, '*.txt'))
if sample_files:
    with open(sample_files[0], 'r') as f:
        print(f"파일명: {os.path.basename(sample_files[0])}")
        print("내용 (첫 2줄만):")
        lines = f.readlines()
        for line in lines[:2]:
            print(line.strip()[:100] + "...") # 너무 기니까 앞부분만 출력
            # 숫자가 엄청 많으면 성공, 4~5개면 실패(Box)
else:
    print("생성된 라벨 파일이 없습니다.")

파일명: IMG_20191215_111114.txt
내용 (첫 2줄만):
2 0.518230 0.330490 0.515970 0.305060 0.515970 0.283030 0.528020 0.261560 0.542330 0.246310 0.561920...
1 0.785630 0.401670 0.805220 0.405060 0.820280 0.405060 0.839870 0.400540 0.857940 0.392630 0.879790...


In [9]:
from ultralytics import YOLO
import torch
import os

# 1. GPU 및 경로 확인
print(f"🔥 사용 중인 GPU: {torch.cuda.get_device_name(0)}")
base_dir = '/content/drive/MyDrive/cv2/Laboro_Tomato'
yaml_path = os.path.join(base_dir, 'laboro_config.yaml')

# yaml 파일 존재 확인
if not os.path.exists(yaml_path):
    raise FileNotFoundError(f"설정 파일이 없습니다: {yaml_path}. 이전 단계에서 yaml 생성을 확인하세요.")

# 2. 모델 로드 (Segmentation Pretrained)
model = YOLO('yolov8m-seg.pt')

# 3. 학습 시작 (A100 최적화 설정)
# - batch: 32 ~ 64 (A100 VRAM 여유에 따라 조정, OOM 발생 시 16으로 감소)
# - workers: 8 (데이터 로딩 병목 방지)
# - cache: True (RAM에 이미지 캐싱하여 속도 극대화)
# - imgsz: 640 (기본값) -> 더 높은 정확도가 필요하면 1024로 변경 가능하나 속도 느려짐
results = model.train(
    data=yaml_path,
    epochs=100,              # 충분히 학습 (patience로 조기 종료됨)
    patience=15,             # 15 epoch 동안 성능 향상 없으면 중단
    batch=32,                # A100 권장 (메모리 부족 시 줄일 것)
    imgsz=640,               # 이미지 크기
    device=0,
    workers=16,              # Colab Pro+ CPU 코어 활용
    optimizer='AdamW',       # 최신 옵티마이저
    lr0=0.001,               # 초기 학습률
    project='/content/drive/MyDrive/cv2/runs',  # 저장 경로
    name='tomato_seg_v8m_final', # 프로젝트 이름
    exist_ok=True,           # 덮어쓰기 허용
    cache=True,              # RAM 캐싱 사용 (학습 속도 대폭 향상)
    cos_lr=True              # Cosine Learning Rate Scheduler 사용
)

print(f"✅ 학습 완료. 모델 저장 경로: {results.save_dir}")

🔥 사용 중인 GPU: NVIDIA A100-SXM4-80GB
Ultralytics 8.4.6 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/cv2/Laboro_Tomato/laboro_config.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=tomato_seg_v8m_final, nbs=64, nms=False, opset=None,